<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s Overal Message:
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [5]:
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [6]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-31 03:24:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-31 03:24:31 (60.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [7]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [8]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-31 03:24:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 172.217.13.240, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-08-31 03:24:31 (90.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [9]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [10]:
train_data = pd.read_csv('train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [11]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
test_data = pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [13]:
random_data = train_data.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
#DW Hundreds feared drowned as migrant boat capsizes off Libya: Hundreds of migrants areÛ_ http://t.co/VOX99FWKcX : This is a Disaster tweet
------------------------------------

tweet number 2 :
That took way longer than I expected : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
@Willie_Am_I @JusttheBottle I would cry in to my nuclear reactor cooling tank!  #winechat : This is NOT Disaster tweet
------------------------------------

tweet number 4 :
2015 new fashion ladies gold watch waterproof WeiQin famous brand michel quartz de lujo caÛ_ http://t.co/1JgsioUJaS http://t.co/719TZEyHFn : This is NOT Disaster tweet
------------------------------------

tweet number 5 :
The Latest: More homes razed by Northern California wildfire - http://t.co/P3g3bQBczu http://t.co/RpBxdfnx5k : This is a Disaster tweet
------------------------------------



## Extract validation data from train data:

In [14]:
train_data['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_data['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [16]:
train_data, val_data, train_label, val_label = train_test_split(train_data['text'].to_numpy(), train_data['target'].to_numpy(), 
                                                                test_size=0.15, 
                                                                random_state=42)

In [17]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [18]:
sum([len(tweet.split()) for tweet in train_data])

96401

In [19]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [20]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [21]:
text_vectorizer.adapt(train_data)

In [22]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorizer(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  736,    9,    1,    8,  161,  681,    5, 1732,   12,    0,
          0,    0,    0,    0])>

In [23]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorizer(random_tweet))

tweet:  Don't miss Chris #Appy's detonation of the myths obscuring our crime in #Hiroshima 70 yrs ago this month. @salon http://t.co/DlP8kPkt2k 


vectorized tweet:  tf.Tensor(
[  63  952 3853    1  531    6    2    1 9862  113 1366    4  123  323
 1834], shape=(15,), dtype=int64)


In [24]:
#unique words
unique_words = text_vectorizer.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['nkulw', 'njenga', 'nj36', 'nittys', 'nitroglycerin', 'nitishkumar', 'nite', 'nissannews', 'nissan', 'niqqa']


## Embedding:

In [25]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim =32, input_length=None)

In [26]:
sample_embed = embedding(text_vectorizer(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[-5.96786663e-03,  1.61555447e-02, -3.62438187e-02,
        -5.09663671e-03,  3.77636589e-02,  1.99260227e-02,
         5.76458126e-03, -2.74267439e-02,  3.43073159e-04,
        -1.84961408e-03, -3.93162742e-02, -4.38921563e-02,
        -1.70133337e-02,  7.94889778e-03, -2.32206658e-03,
        -2.75335796e-02, -2.64940858e-02,  4.36343439e-02,
        -1.34332553e-02, -3.74578461e-02,  4.10373919e-02,
        -3.52696888e-02, -2.01918017e-02,  1.46185048e-02,
         3.35670635e-03, -3.11732050e-02, -3.35509405e-02,
         3.65964063e-02,  3.53174098e-02,  1.33813284e-02,
         4.81343605e-02,  3.94864343e-02],
       [-6.86535984e-03,  4.25418466e-03, -4.17943224e-02,
         2.61981525e-02, -4.14109230e-02,  5.16176224e-05,
        -2.64995825e-02,  4.20099497e-03, -4.36757579e-02,
         2.34754123e-02,  3.50993313e-02, -3.82528305e-02,
        -3.07537448e-02, -2.39195358e-02,  1.92391016e-02,
        -2.15329882e-0

## Analyze with Naive Bayes using TF-IDF:

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [28]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
val_pred = model_0.predict(val_data)
val_pred

array([0, 0, 0, ..., 1, 0, 0])

In [30]:
val_label

array([1, 0, 1, ..., 1, 0, 0])

In [31]:
accuracy_score(val_label, val_pred)

0.7863397548161121

In [32]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       651
           1       0.85      0.61      0.71       491

    accuracy                           0.79      1142
   macro avg       0.80      0.76      0.77      1142
weighted avg       0.80      0.79      0.78      1142



## Analyze with Vanilla Neural Network:

In [33]:
from helper_functions import create_tensorboard_callback

In [34]:
inputs = tf.keras.layers.Input(shape= (1,), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.keras.layers.Dense(10, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs, name='ANN')

In [35]:
model_1.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 1)                 11      

In [36]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [37]:
train_label.shape

(6471,)

In [38]:
train_data.shape

(6471,)

In [39]:
# !rm -rf './tensorboard/ANN'

In [40]:
history_1 = model_1.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220831-032433
Epoch 1/10
203/203 [==============================] - 3s 8ms/step - loss: 0.6325 - accuracy: 0.6688 - val_loss: 0.5533 - val_accuracy: 0.7811
Epoch 2/10
203/203 [==============================] - 1s 7ms/step - loss: 0.4465 - accuracy: 0.8224 - val_loss: 0.4668 - val_accuracy: 0.8030
Epoch 3/10
203/203 [==============================] - 2s 8ms/step - loss: 0.3255 - accuracy: 0.8720 - val_loss: 0.4667 - val_accuracy: 0.7933
Epoch 4/10
203/203 [==============================] - 1s 7ms/step - loss: 0.2506 - accuracy: 0.9057 - val_loss: 0.4895 - val_accuracy: 0.7846
Epoch 5/10
203/203 [==============================] - 1s 7ms/step - loss: 0.1968 - accuracy: 0.9288 - val_loss: 0.5219 - val_accuracy: 0.7898
Epoch 6/10
203/203 [==============================] - 1s 7ms/step - loss: 0.1573 - accuracy: 0.9434 - val_loss: 0.5657 - val_accuracy: 0.7776
Epoch 7/10
203/203 [==============================] - 1s 7ms/step - loss: 0.1272 - 

In [41]:
model_1.evaluate(val_data,val_label)

36/36 [==============================] - 0s 2ms/step - loss: 0.7545 - accuracy: 0.7706


[0.754477858543396, 0.7705779075622559]

In [42]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [43]:
model_0.predict_proba(tweets)

array([[0.20651409, 0.79348591],
       [0.58784876, 0.41215124],
       [0.70686811, 0.29313189]])

In [44]:
model_0.predict(tweets)

array([1, 0, 0])

In [45]:
model_1.predict(tweets)

array([[0.99287045],
       [0.88256466],
       [0.14784294]], dtype=float32)

## Retrieve the trained word embeddings:

In [47]:
weights = model_1.get_layer('embedding').get_weights()[0]
weights

array([[ 6.72750473e-02, -1.76332802e-01, -5.75478747e-02, ...,
        -8.55744630e-02,  3.81814130e-02, -5.85654303e-02],
       [ 2.64278296e-02, -2.69873887e-01, -1.02824077e-03, ...,
        -6.11876994e-02,  5.99630997e-02, -6.99323714e-02],
       [ 3.34783010e-02, -1.62255123e-01, -6.51154760e-03, ...,
        -5.86111173e-02,  5.27028628e-02, -3.73361073e-02],
       ...,
       [-9.25147235e-02,  1.03823304e-01,  1.30998969e-01, ...,
         1.04183078e-01,  1.57411963e-01,  1.66003808e-01],
       [ 1.20862693e-01, -1.22856744e-01, -7.78003335e-02, ...,
        -1.08233266e-01, -1.08015358e-01, -7.39322603e-02],
       [ 5.19361565e-05, -1.34559916e-02, -5.54656312e-02, ...,
        -5.95826246e-02, -4.73720282e-02, -4.97975610e-02]], dtype=float32)

In [48]:
weights.shape

(10000, 32)

In [49]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [50]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [51]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Then, use [Embedding Projector](http://projector.tensorflow.org/) to Visualize the embeddings

## Analyzie with LSTM model:

In [52]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.LSTM(64,  return_sequences=True)(x)
x = tf.keras.layers.LSTM(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs, name='LSTM')

In [53]:
model_2.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            24832     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                              

In [54]:
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [55]:
# !rm -rf './tensorboard/RNN'

In [56]:
history_2 = model_2.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','LSTM')])

Saving TensorBoard log files to: tensorboard/LSTM/20220831-035618
Epoch 1/10
203/203 [==============================] - 9s 25ms/step - loss: 0.1708 - accuracy: 0.9278 - val_loss: 0.8310 - val_accuracy: 0.7294
Epoch 2/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0790 - accuracy: 0.9683 - val_loss: 0.9363 - val_accuracy: 0.7706
Epoch 3/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0638 - accuracy: 0.9726 - val_loss: 0.9658 - val_accuracy: 0.7426
Epoch 4/10
203/203 [==============================] - 4s 19ms/step - loss: 0.0550 - accuracy: 0.9793 - val_loss: 1.3280 - val_accuracy: 0.7356
Epoch 5/10
203/203 [==============================] - 4s 21ms/step - loss: 0.0474 - accuracy: 0.9791 - val_loss: 1.4665 - val_accuracy: 0.7592
Epoch 6/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0445 - accuracy: 0.9804 - val_loss: 1.3453 - val_accuracy: 0.7601
Epoch 7/10
203/203 [==============================] - 4s 21ms/step - loss: 0

In [57]:
model_2.evaluate(val_data,val_label)

36/36 [==============================] - 0s 5ms/step - loss: 1.4876 - accuracy: 0.7557


[1.4876171350479126, 0.7556917667388916]

### custom tweets:

In [58]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [59]:
model_2.predict(tweets)

array([[9.9969971e-01],
       [9.9680465e-01],
       [4.8527122e-04]], dtype=float32)

## Analyzie with GRU model:

In [60]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.GRU(64,  return_sequences=True)(x)
x = tf.keras.layers.GRU(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs, name='GRU')

In [61]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 gru (GRU)                   (None, 15, 64)            18816     
                                                                 
 gru_1 (GRU)                 (None, 32)                9408      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                               

In [62]:
model_3.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [63]:
# !rm -rf './tensorboard/RNN'

In [64]:
history_3 = model_3.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','GRU')])

Saving TensorBoard log files to: tensorboard/GRU/20220831-035745
Epoch 1/10
203/203 [==============================] - 9s 23ms/step - loss: 0.1365 - accuracy: 0.9499 - val_loss: 0.8891 - val_accuracy: 0.7636
Epoch 2/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0569 - accuracy: 0.9790 - val_loss: 0.8330 - val_accuracy: 0.7636
Epoch 3/10
203/203 [==============================] - 4s 21ms/step - loss: 0.0512 - accuracy: 0.9801 - val_loss: 1.0651 - val_accuracy: 0.7601
Epoch 4/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0410 - accuracy: 0.9804 - val_loss: 1.3184 - val_accuracy: 0.7504
Epoch 5/10
203/203 [==============================] - 5s 26ms/step - loss: 0.0425 - accuracy: 0.9804 - val_loss: 1.2942 - val_accuracy: 0.7601
Epoch 6/10
203/203 [==============================] - 4s 19ms/step - loss: 0.0384 - accuracy: 0.9825 - val_loss: 1.5536 - val_accuracy: 0.7487
Epoch 7/10
203/203 [==============================] - 4s 20ms/step - loss: 0.

In [65]:
model_3.evaluate(val_data,val_label)

36/36 [==============================] - 0s 4ms/step - loss: 1.9333 - accuracy: 0.7443


[1.9332919120788574, 0.7443082332611084]

### custom tweets:

In [66]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [67]:
model_3.predict(tweets)

array([[9.9997616e-01],
       [9.9991935e-01],
       [4.3305921e-05]], dtype=float32)

## Analyzie with Bidirectional LSTM model:

In [68]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)


x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

In [69]:
model_4.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                

In [70]:
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [71]:
# !rm -rf './tensorboard/RNN'

In [72]:
history_4 = model_4.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-035835
Epoch 1/10
203/203 [==============================] - 15s 39ms/step - loss: 0.1070 - accuracy: 0.9618 - val_loss: 0.8702 - val_accuracy: 0.7539
Epoch 2/10
203/203 [==============================] - 7s 33ms/step - loss: 0.0357 - accuracy: 0.9825 - val_loss: 1.3998 - val_accuracy: 0.7513
Epoch 3/10
203/203 [==============================] - 6s 32ms/step - loss: 0.0367 - accuracy: 0.9830 - val_loss: 1.8597 - val_accuracy: 0.7137
Epoch 4/10
203/203 [==============================] - 8s 38ms/step - loss: 0.0354 - accuracy: 0.9824 - val_loss: 1.6636 - val_accuracy: 0.7408
Epoch 5/10
203/203 [==============================] - 8s 38ms/step - loss: 0.0307 - accuracy: 0.9844 - val_loss: 1.6903 - val_accuracy: 0.7566
Epoch 6/10
203/203 [==============================] - 7s 34ms/step - loss: 0.0278 - accuracy: 0.9856 - val_loss: 2.0564 - val_accuracy: 0.7399
Epoch 7/10
203/203 [==============================] - 7s 34ms

In [73]:
model_4.evaluate(val_data,val_label)

36/36 [==============================] - 0s 8ms/step - loss: 1.7516 - accuracy: 0.7504


[1.7515912055969238, 0.75043785572052]

### custom tweets:

In [74]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [75]:
model_4.predict(tweets)

array([[9.9998093e-01],
       [9.9994528e-01],
       [1.1833711e-04]], dtype=float32)

## Analyzie with Convolutional 1D model:

In [76]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)
print(x.shape)

x = tf.keras.layers.Conv1D(filters=64, kernel_size=3,padding= 'valid')(x)
print(x.shape)
x = tf.keras.layers.Conv1D(filters=128, kernel_size=3,padding= 'valid')(x)
print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

(None, 15, 32)
(None, 13, 64)
(None, 11, 128)


In [77]:
model_5.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 conv1d (Conv1D)             (None, 13, 64)            6208      
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 128)           24704     
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                       

In [78]:
model_5.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [79]:
# !rm -rf './tensorboard/RNN'

In [80]:
history_5 = model_5.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-035955
Epoch 1/10
203/203 [==============================] - 3s 11ms/step - loss: 0.1321 - accuracy: 0.9566 - val_loss: 0.8825 - val_accuracy: 0.7539
Epoch 2/10
203/203 [==============================] - 2s 10ms/step - loss: 0.0606 - accuracy: 0.9745 - val_loss: 1.1966 - val_accuracy: 0.7329
Epoch 3/10
203/203 [==============================] - 2s 10ms/step - loss: 0.0426 - accuracy: 0.9802 - val_loss: 1.5662 - val_accuracy: 0.7294
Epoch 4/10
203/203 [==============================] - 2s 10ms/step - loss: 0.0374 - accuracy: 0.9825 - val_loss: 1.6455 - val_accuracy: 0.7408
Epoch 5/10
203/203 [==============================] - 2s 9ms/step - loss: 0.0370 - accuracy: 0.9813 - val_loss: 1.8228 - val_accuracy: 0.7426
Epoch 6/10
203/203 [==============================] - 2s 10ms/step - loss: 0.0336 - accuracy: 0.9828 - val_loss: 1.8145 - val_accuracy: 0.7312
Epoch 7/10
203/203 [==============================] - 3s 13ms/s

In [81]:
model_5.evaluate(val_data,val_label)

36/36 [==============================] - 0s 6ms/step - loss: 2.0272 - accuracy: 0.7496


[2.027242422103882, 0.74956214427948]

### custom tweets:

In [82]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [83]:
model_5.predict(tweets)

array([[0.9974623 ],
       [0.96732557],
       [0.00154772]], dtype=float32)

## Analyzie with Universal Sentence Encoder tensorflow hub:

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed(["The quick brown fox jumps over the lazy dog.",
                      "I am a sentence for which I would like to get its embedding"])

print(embed_sample)

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


In [89]:
embed(train_data)

<tf.Tensor: shape=(6471, 512), dtype=float32, numpy=
array([[-0.04645307, -0.01459127,  0.06717625, ...,  0.03142272,
        -0.02836458, -0.07366095],
       [-0.00339469, -0.01426822,  0.04503542, ..., -0.0444056 ,
        -0.06956182, -0.01792216],
       [ 0.0456574 , -0.02573171,  0.03533637, ..., -0.06532598,
        -0.03026031,  0.00103621],
       ...,
       [-0.03393722, -0.0616745 ,  0.0497893 , ..., -0.05219646,
        -0.05812604, -0.0630734 ],
       [ 0.02153323,  0.06060664, -0.07061632, ..., -0.04428504,
        -0.00652967, -0.00850326],
       [ 0.0264951 , -0.05326021, -0.05051778, ..., -0.01227471,
         0.01156287, -0.00953934]], dtype=float32)>

In [99]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               trainable=False,
                               dtype=tf.string,
                               name= 'USE')

In [116]:
model_6 = tf.keras.Sequential([
    encoder_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')],
    name = 'USE_pretrained_model'
)

In [117]:
model_6.summary()

Model: "USE_pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_13 (Dense)            (None, 64)                32832     
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 34,945
Non-trainable params: 256,797,824
_________________________________________________________________


In [118]:
model_6.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [119]:
# !rm -rf './tensorboard/RNN'

In [120]:
history_6 = model_6.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','USE_pretrained_model')])

Saving TensorBoard log files to: tensorboard/USE_pretrained_model/20220831-051437
Epoch 1/10
203/203 [==============================] - 4s 12ms/step - loss: 0.4986 - accuracy: 0.7724 - val_loss: 0.4232 - val_accuracy: 0.8126
Epoch 2/10
203/203 [==============================] - 2s 10ms/step - loss: 0.4080 - accuracy: 0.8192 - val_loss: 0.4169 - val_accuracy: 0.8240
Epoch 3/10
203/203 [==============================] - 2s 10ms/step - loss: 0.3872 - accuracy: 0.8323 - val_loss: 0.4057 - val_accuracy: 0.8310
Epoch 4/10
203/203 [==============================] - 2s 10ms/step - loss: 0.3656 - accuracy: 0.8450 - val_loss: 0.4167 - val_accuracy: 0.8266
Epoch 5/10
203/203 [==============================] - 2s 10ms/step - loss: 0.3424 - accuracy: 0.8513 - val_loss: 0.4102 - val_accuracy: 0.8284
Epoch 6/10
203/203 [==============================] - 2s 10ms/step - loss: 0.3123 - accuracy: 0.8677 - val_loss: 0.4254 - val_accuracy: 0.8187
Epoch 7/10
203/203 [==============================] - 2s 10m

In [125]:
model_6.evaluate(val_data,val_label)

36/36 [==============================] - 0s 11ms/step - loss: 0.5156 - accuracy: 0.8100


[0.5156313180923462, 0.8099824786186218]

### custom tweets:

In [130]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [131]:
model_6.predict(tweets)

array([[9.9973571e-01],
       [3.3517218e-01],
       [7.2938204e-04]], dtype=float32)

## Finding most wrong predicted tweets: